# Reproduction of First-principles analysis of cross-resonance gate operation

Malekakhlagh et al. PRA 102, 042605

Note that the drive frequency is set to the undressed target qubit frequency, which results in $\mathcal{O}(J^2)$ differences in the calculation results compared to ideal experimental observations.

In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
import time
import pickle
import copy
import logging
from collections import defaultdict
from multiprocessing import Pool
from IPython.display import Math
from sympy import Add, Expr, I, Mul, Pow, S, Symbol, diff, pi, preorder_traversal, simplify, latex
from sympy.physics.quantum import Commutator, IdentityOperator, HermitianOperator, TensorProduct, qapply
from symqudit.two_transmon_hamiltonian import (TwoTransmonHamiltonian, FrequencyDiff, sort_block_diagonal,
                                               to_dict, from_dict, dict_product)
from symqudit.schrieffer_wolff_expansion import SWExpansion, integrate_exp_term, integrate_expr
from symqudit.common import ketbra, get_expr_at_order, organize_by_denom, DiscreteBra, DiscreteKet, DiscreteOuterProduct

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

In [2]:
Id = IdentityOperator()
tvar = Symbol('t', real=True)

def expand_opprod(lhs, rhs, blkdiag_only=False) -> dict[tuple, Expr]:
    start = time.time()
    op_prod = dict_product(op_dicts[lhs], op_dicts[rhs],
                           blkdiag_only=blkdiag_only, expand=False)

    if blkdiag_only:
        filtered = {}
        for ket, bra_dict in op_prod.items():
            filtered[ket] = {}
            for bra, coeff in bra_dict.items():
                coeff = coeff.expand()
                if isinstance(coeff, Add):
                    cterms = coeff.args
                else:
                    cterms = [coeff]
                cterms = [cterm for cterm in cterms if tvar not in preorder_traversal(cterm)]
                if cterms:
                    filtered[ket][bra] = Add(*cterms)

            if not filtered[ket]:
                filtered.pop(ket)

        op_prod = filtered
        
    end = time.time()
    logger.info('dict_product(%s, %s) in %f seconds', lhs, rhs, end - start)

    return op_prod


def calculate_commutator(fore, back):
    result = copy.deepcopy(fore)
    for ket, bra_dict in back.items():
        if ket not in result:
            result[ket] = defaultdict(lambda: S.Zero)
        for bra, coeff in bra_dict.items():
            result[ket][bra] -= coeff

    return result


def expand_expr(expr):
    return expr.expand()


def sort_static(coeff):
    coeff = coeff.expand()
    if isinstance(coeff, Add):
        terms = coeff.args
    else:
        terms = [coeff]

    s_terms = []
    d_terms = []
    
    for term in terms:
        if tvar not in preorder_traversal(term):
            s_terms.append(term)
        else:
            d_terms.append(term)

    if s_terms:
        s_terms = Add(*s_terms)
    else:
        s_terms = None
    if d_terms:
        d_terms = Add(*d_terms)
    else:
        d_terms = None

    return s_terms, d_terms

In [3]:
def from_dict_parallel(terms):
    ops = []
    coeffs = []
    
    for ket, bra_dict in terms.items():
        for bra, coeff in bra_dict.items():
            coeffs.append(coeff)
            if ket[1] is None:
                ops.append(TensorProduct(ketbra(ket[0], bra[0]), Id))
            else:
                ops.append(TensorProduct(ketbra(ket[0], bra[0]), ketbra(ket[1], bra[1])))
                
    with Pool() as pool:
        coeffs = pool.map(expand_expr, coeffs)

    return Add(*[c * o for c, o in zip(coeffs, ops)])


def sort_block_diagonal_parallel(all_terms):
    maybe_blkdiag = []
    nonblkdiag = defaultdict(dict)

    for ket, bra_dict in all_terms.items():
        for bra, coeff in bra_dict.items():
            if ket[0] == bra[0]:
                maybe_blkdiag.append((ket, bra, coeff))
            else:
                nonblkdiag[ket][bra] = coeff

    blkdiag = defaultdict(dict)
    if maybe_blkdiag:
        with Pool() as pool:
            sorted_coeffs = pool.map(sort_static, [c for _, _, c in maybe_blkdiag])

        for (s_terms, d_terms), (ket, bra, _) in zip(sorted_coeffs, maybe_blkdiag):
            if s_terms is not None:
                blkdiag[ket][bra] = s_terms
            if d_terms is not None:
                nonblkdiag[ket][bra] = d_terms

    return blkdiag, nonblkdiag


def integrate_expr_parallel(nonblkdiag):
    flattened = [(ket, bra, coeff) for ket, bra_dict in nonblkdiag.items() for bra, coeff in bra_dict.items()]
    with Pool() as pool:
        coeffs = pool.map(expand_expr, [coeff for _, _, coeff in flattened])
    
    coeff_terms = []
    term_bounds = [0]
    for coeff in coeffs:
        if isinstance(coeff, Add):
            coeff_terms += list(coeff.args)
        else:
            coeff_terms.append(coeff)
        term_bounds.append(len(coeff_terms))

    with Pool() as pool:
        int_terms = pool.map(integrate_exp_term, coeff_terms)
    
    integrated = defaultdict(dict)
    for iterm, (ket, bra, _) in enumerate(flattened):
        integrated[ket][bra] = Add(*int_terms[term_bounds[iterm]:term_bounds[iterm + 1]])

    return integrated

In [4]:
cutoff = 4
c_params = (100., 0.1)
t_params = (104., 0.1)

In [5]:
tth = TwoTransmonHamiltonian((c_params[0], t_params[0]), (c_params[1], t_params[1]))

h_dirac = tth.h_dirac(cutoff=cutoff).expand()
h_dirac = h_dirac.subs({tth.drive_freq: tth.qt.symbolic_eigenvalue(1).doit(), tth.drive_phase: pi})
h_dirac = tth.subs_delta(h_dirac).expand()
h_dirac = tth.symbolify(h_dirac)

int_scale = Symbol('lambda', real=True, nonnegative=True)
h_i = HermitianOperator('H_I')

In [6]:
h_dirac

J*{\nu^c}_{0,1}*{\nu^t}_{0,1}*exp(I*t*{\Delta^{1}_{1}})*|1c><0c|x|0t><1t| + J*{\nu^c}_{0,1}*{\nu^t}_{0,1}*exp(-I*t*{\Delta^{1}_{1}})*|0c><1c|x|1t><0t| + J*{\nu^c}_{0,1}*{\nu^t}_{1,2}*exp(I*t*{\Delta^{1}_{2}})*|1c><0c|x|1t><2t| + J*{\nu^c}_{0,1}*{\nu^t}_{1,2}*exp(-I*t*{\Delta^{1}_{2}})*|0c><1c|x|2t><1t| + J*{\nu^c}_{0,1}*{\nu^t}_{2,3}*exp(I*t*{\Delta^{1}_{3}})*|1c><0c|x|2t><3t| + J*{\nu^c}_{0,1}*{\nu^t}_{2,3}*exp(-I*t*{\Delta^{1}_{3}})*|0c><1c|x|3t><2t| + J*{\nu^c}_{0,1}*{\nu^t}_{3,4}*exp(I*t*{\Delta^{1}_{4}})*|1c><0c|x|3t><4t| + J*{\nu^c}_{0,1}*{\nu^t}_{3,4}*exp(-I*t*{\Delta^{1}_{4}})*|0c><1c|x|4t><3t| + J*{\nu^c}_{1,2}*{\nu^t}_{0,1}*exp(I*t*{\Delta^{2}_{1}})*|2c><1c|x|0t><1t| + J*{\nu^c}_{1,2}*{\nu^t}_{0,1}*exp(-I*t*{\Delta^{2}_{1}})*|1c><2c|x|1t><0t| + J*{\nu^c}_{1,2}*{\nu^t}_{1,2}*exp(I*t*{\Delta^{2}_{2}})*|2c><1c|x|1t><2t| + J*{\nu^c}_{1,2}*{\nu^t}_{1,2}*exp(-I*t*{\Delta^{2}_{2}})*|1c><2c|x|2t><1t| + J*{\nu^c}_{1,2}*{\nu^t}_{2,3}*exp(I*t*{\Delta^{2}_{3}})*|2c><1c|x|2t><3t| + J*{\nu

In [7]:
def calc_heff_at_order(order, blkdiag_only=False):
    comm_expr = get_expr_at_order(expansion, int_scale, order) + sw.gdot_n[order]

    start = time.time()
    if order == 1:
        # No commutator for Order λ -> just substitute H_I
        states_expr[order] = comm_expr.subs({h_i: h_dirac})
        all_terms = to_dict(states_expr[order])

    else:
        while True:
            if isinstance(comm_expr, Add):
                terms = comm_expr.args
            else:
                terms = [comm_expr]
            for term in terms:
                comm = term.args_cnc()[1][0]
                if isinstance(comm.args[0], Commutator) or isinstance(comm.args[1], Commutator):
                    break
            else:
                break
            new_expr = comm_expr.subs(commutator_subs)
            if new_expr == comm_expr:
                raise RuntimeError('Cannot reduce commutator %s', comm_expr)
            comm_expr = new_expr

        coeffs = []
        commutators = []
        opprod_terms = []
        for term in terms:
            c, nc = term.args_cnc()
            coeff = Mul(*c)
            comm = nc[0]
            if isinstance(comm, Mul):
                coeff *= comm.args[0]
                comm = comm.args[1]
            coeffs.append(coeff)
            commutators.append(comm)
            opprod_terms.append((comm.args[0], comm.args[1]))
            opprod_terms.append((comm.args[1], comm.args[0]))
        
        with Pool() as pool:
            if blkdiag_only:
                args = [arg + (True,) for arg in opprod_terms]
            else:
                args = opprod_terms
            opprods = pool.starmap(expand_opprod, args)
            
        end = time.time()
        logger.info('expand_opprod for order %d in %f seconds', order, end - start)
        start = end
        
        comm_dicts_source = list(zip(opprods[0:-1:2], opprods[1::2]))
        with Pool() as pool:
            comm_dicts = pool.starmap(calculate_commutator, comm_dicts_source)
            
        end = time.time()
        logger.info('calculate_commutator for order %d in %f seconds', order, end - start)
        start = end
        
        all_terms = defaultdict(lambda: defaultdict(lambda: S.Zero))
        for coeff, comm, comm_dict in zip(coeffs, commutators, comm_dicts):
            if not blkdiag_only:
                placeholder = HermitianOperator(f'C_{len(commutator_subs)}')
                commutator_subs[comm] = placeholder
                op_dicts[placeholder] = comm_dict

            for ket, bra_dict in comm_dict.items():
                for bra, c in bra_dict.items():
                    all_terms[ket][bra] += coeff * c
                
        end = time.time()
        logger.info('all_terms compilation for order %d in %f seconds', order, end - start)
        start = end

        # Expression in terms of TensorProduct(OuterProduct, OuterProduct or Identity)
        states_expr[order] = from_dict_parallel(all_terms)

    if blkdiag_only:
        hieff[order] = states_expr[order]
        return

    blkdiag, nonblkdiag = sort_block_diagonal_parallel(all_terms)
    hieff[order] = from_dict_parallel(blkdiag)
    gdot[order] = from_dict_parallel(nonblkdiag)
    
    end = time.time()
    logger.info('sort_block_diagonal(%d) in %f seconds', order, end - start)
    start = end
    
    integrated = integrate_expr_parallel(nonblkdiag)
    g[order] = from_dict_parallel(integrated)
    
    end = time.time()
    logger.info('integrate_expr(%d) in %f seconds', order, end - start)

    op_dicts.update({sw.g_n[order]: to_dict(g[order]), sw.gdot_n[order]: to_dict(gdot[order])})

In [8]:
op_dicts = {h_i: to_dict(h_dirac)}
states_expr = {}
commutator_subs = {}
g = {}
gdot = {}
hieff = {}

sw = SWExpansion()
expansion = sw.expand(int_scale * h_i, int_scale, 4)

## Order λ
calc_heff_at_order(1)

# Special substitution
expansion = expansion.subs({sw.gdot_n[1]: h_i})

## Order λ^2
calc_heff_at_order(2)

INFO:root:sort_block_diagonal(1) in 1.617250 seconds
INFO:root:integrate_expr(1) in 2.412985 seconds
INFO:root:dict_product(G_1, H_I) in 0.018726 seconds
INFO:root:dict_product(H_I, G_1) in 0.028255 seconds
INFO:root:expand_opprod for order 2 in 0.811177 seconds
INFO:root:calculate_commutator for order 2 in 0.795508 seconds
INFO:root:all_terms compilation for order 2 in 0.008548 seconds
INFO:root:sort_block_diagonal(2) in 4.840205 seconds
INFO:root:integrate_expr(2) in 5.191500 seconds


In [9]:
compos = tth.pauli_components(hieff[2], 2, 2)
pauli_names = ['I', 'X', 'Y', 'Z']

for idx in [(0, 1), (3, 0), (3, 1), (3, 3)]:
    display(Math(fr'\omega_{{{pauli_names[idx[0]]}{pauli_names[idx[1]]}}}^{{(2)}} = ' + latex(compos[idx])))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [10]:
# Order λ^3
calc_heff_at_order(3)

INFO:root:dict_product(G_1, \dot{G}_2) in 0.022992 seconds
INFO:root:dict_product(\dot{G}_2, G_1) in 0.025730 seconds
INFO:root:dict_product(G_1, C_0) in 0.034861 seconds
INFO:root:dict_product(G_2, H_I) in 0.046166 seconds
INFO:root:dict_product(H_I, G_2) in 0.057194 seconds
INFO:root:dict_product(C_0, G_1) in 0.396055 seconds
INFO:root:expand_opprod for order 3 in 1.951927 seconds
INFO:root:calculate_commutator for order 3 in 2.058219 seconds
INFO:root:all_terms compilation for order 3 in 0.072147 seconds
INFO:root:sort_block_diagonal(3) in 38.637344 seconds
INFO:root:integrate_expr(3) in 15.472812 seconds


In [11]:
calc_heff_at_order(4)

INFO:root:dict_product(H_I, G_3) in 0.000269 seconds
INFO:root:dict_product(G_3, H_I) in 0.000238 seconds
INFO:root:dict_product(G_1, \dot{G}_3) in 0.039698 seconds
INFO:root:dict_product(G_1, C_3) in 0.056959 seconds
INFO:root:dict_product(G_2, \dot{G}_2) in 0.056076 seconds
INFO:root:dict_product(G_2, C_0) in 0.059399 seconds
INFO:root:dict_product(G_1, C_2) in 0.065222 seconds
INFO:root:dict_product(G_1, C_1) in 0.064779 seconds
INFO:root:dict_product(\dot{G}_3, G_1) in 0.065972 seconds
INFO:root:dict_product(\dot{G}_2, G_2) in 0.067768 seconds
INFO:root:dict_product(C_0, G_2) in 0.418097 seconds
INFO:root:dict_product(C_3, G_1) in 0.437137 seconds
INFO:root:dict_product(C_1, G_1) in 0.478005 seconds
INFO:root:dict_product(C_2, G_1) in 1.376780 seconds
INFO:root:expand_opprod for order 4 in 13.215232 seconds
INFO:root:calculate_commutator for order 4 in 13.165063 seconds
INFO:root:all_terms compilation for order 4 in 0.654274 seconds
INFO:root:sort_block_diagonal(4) in 1006.087054 s

In [12]:
compos = tth.pauli_components(hieff[4], 2, 2)

In [15]:
f = tth.symbolify(tth.coupling * tth.drive_amp ** 3 * organize_by_denom((get_expr_at_order(compos[3, 1], tth.drive_amp, 3) / tth.coupling).expand()))


In [16]:
Math(r'\omega_{ZX}^{(4)} = ' + latex(f))

<IPython.core.display.Math object>

In [42]:
s = list(f.free_symbols)

In [43]:
s

[{\nu^c}_{1,2},
 {\nu^c}_{0,1},
 {\nu^t}_{0,1},
 {\Delta^{1}_{1}},
 {\nu^c}_{2,3},
 {\Delta^{2}_{1}},
 {\Delta^{3}_{1}}]

In [ ]:
hieff4_omega3 = get_expr_at_order(hieff[4].expand(), tth.drive_amp, 3)

In [52]:
nuc12 = tth.symbolify(tth.qc._transition_amp(1, 2))
nuc23 = tth.symbolify(tth.qc._transition_amp(2, 3))
nut01 = tth.symbolify(tth.qt._transition_amp(0, 1))
d11 = tth.symbolify(FrequencyDiff(1, 1))
for arg in hieff4_omega3.args:
    symb = arg.free_symbols
    if tvar not in symb and nuc12 in symb and nuc23 in symb and nut01 in symb:
        display(arg)

J*{\nu^c}_{1,2}**2*{\nu^c}_{2,3}**2*{\nu^t}_{0,1}*|1c><1c|x|0t><1t|/(16*{\Delta^{2}_{1}}**3 + 16*{\Delta^{2}_{1}}**2*{\Delta^{3}_{1}})

J*{\nu^c}_{1,2}**2*{\nu^c}_{2,3}**2*{\nu^t}_{0,1}*|2c><2c|x|0t><1t|/(24*{\Delta^{2}_{1}}**3 + 24*{\Delta^{2}_{1}}**2*{\Delta^{3}_{1}})

J*{\nu^c}_{1,2}**2*{\nu^c}_{2,3}**2*{\nu^t}_{0,1}*|2c><2c|x|1t><0t|/(24*{\Delta^{2}_{1}}**3 + 24*{\Delta^{2}_{1}}**2*{\Delta^{3}_{1}})

J*{\nu^c}_{1,2}**2*{\nu^c}_{2,3}**2*{\nu^t}_{0,1}*|3c><3c|x|0t><1t|/(24*{\Delta^{2}_{1}}**3 + 24*{\Delta^{2}_{1}}**2*{\Delta^{3}_{1}})

J*{\nu^c}_{1,2}**2*{\nu^c}_{2,3}**2*{\nu^t}_{0,1}*|3c><3c|x|1t><0t|/(24*{\Delta^{2}_{1}}**3 + 24*{\Delta^{2}_{1}}**2*{\Delta^{3}_{1}})

...

J*{\nu^c}_{1,2}**2*{\nu^c}_{2,3}**2*{\nu^t}_{0,1}*|2c><2c|x|1t><0t|/(16*{\Delta^{2}_{1}}*{\Delta^{3}_{1}}**2)

J*{\nu^c}_{1,2}**2*{\nu^c}_{2,3}**2*{\nu^t}_{0,1}*|1c><1c|x|0t><1t|/(24*{\Delta^{2}_{1}}**2*{\Delta^{3}_{1}})

J*{\nu^c}_{1,2}**2*{\nu^c}_{2,3}**2*{\nu^t}_{0,1}*|1c><1c|x|1t><0t|/(24*{\Delta^{2}_{1}}**2*{\Delta^{3}_{1}})

J*{\nu^c}_{1,2}**2*{\nu^c}_{2,3}**2*{\nu^t}_{0,1}*|3c><3c|x|0t><1t|/(48*{\Delta^{2}_{1}}**2*{\Delta^{3}_{1}})

J*{\nu^c}_{1,2}**2*{\nu^c}_{2,3}**2*{\nu^t}_{0,1}*|3c><3c|x|1t><0t|/(48*{\Delta^{2}_{1}}**2*{\Delta^{3}_{1}})

In [44]:
from sympy import sqrt
simplify(f.subs({s[0]: sqrt(2), s[1]: 1, s[2]: 1, s[3]: d, s[4]: sqrt(3), s[5]: d + a, s[6]: d + 2 * a}))

(16*Delta**7 + 48*Delta**6*alpha - 133*Delta**5*alpha**2 - 799*Delta**4*alpha**3 - 1368*Delta**3*alpha**4 - 1088*Delta**2*alpha**5 - 438*Delta*alpha**6 - 72*alpha**7)/(24*Delta**3*(4*Delta**7 + 36*Delta**6*alpha + 135*Delta**5*alpha**2 + 273*Delta**4*alpha**3 + 321*Delta**3*alpha**4 + 219*Delta**2*alpha**5 + 80*Delta*alpha**6 + 12*alpha**7))

In [45]:
frac

alpha**2*(9*Delta**3 + 15*Delta**2*alpha + 11*Delta*alpha**2 + 3*alpha**3)/(2*Delta**3*(4*Delta**5 + 20*Delta**4*alpha + 39*Delta**3*alpha**2 + 37*Delta**2*alpha**3 + 17*Delta*alpha**4 + 3*alpha**5))

In [17]:
h_dirac

J*{\nu^c}_{0,1}*{\nu^t}_{0,1}*exp(I*t*{\Delta^{1}_{1}})*|1c><0c|x|0t><1t| + J*{\nu^c}_{0,1}*{\nu^t}_{0,1}*exp(-I*t*{\Delta^{1}_{1}})*|0c><1c|x|1t><0t| + J*{\nu^c}_{0,1}*{\nu^t}_{1,2}*exp(I*t*{\Delta^{1}_{2}})*|1c><0c|x|1t><2t| + J*{\nu^c}_{0,1}*{\nu^t}_{1,2}*exp(-I*t*{\Delta^{1}_{2}})*|0c><1c|x|2t><1t| + J*{\nu^c}_{0,1}*{\nu^t}_{2,3}*exp(I*t*{\Delta^{1}_{3}})*|1c><0c|x|2t><3t| + J*{\nu^c}_{0,1}*{\nu^t}_{2,3}*exp(-I*t*{\Delta^{1}_{3}})*|0c><1c|x|3t><2t| + J*{\nu^c}_{0,1}*{\nu^t}_{3,4}*exp(I*t*{\Delta^{1}_{4}})*|1c><0c|x|3t><4t| + J*{\nu^c}_{0,1}*{\nu^t}_{3,4}*exp(-I*t*{\Delta^{1}_{4}})*|0c><1c|x|4t><3t| + J*{\nu^c}_{1,2}*{\nu^t}_{0,1}*exp(I*t*{\Delta^{2}_{1}})*|2c><1c|x|0t><1t| + J*{\nu^c}_{1,2}*{\nu^t}_{0,1}*exp(-I*t*{\Delta^{2}_{1}})*|1c><2c|x|1t><0t| + J*{\nu^c}_{1,2}*{\nu^t}_{1,2}*exp(I*t*{\Delta^{2}_{2}})*|2c><1c|x|1t><2t| + J*{\nu^c}_{1,2}*{\nu^t}_{1,2}*exp(-I*t*{\Delta^{2}_{2}})*|1c><2c|x|2t><1t| + J*{\nu^c}_{1,2}*{\nu^t}_{2,3}*exp(I*t*{\Delta^{2}_{3}})*|2c><1c|x|2t><3t| + J*{\nu

In [ ]:
with open('first_principle_qutrit.pkl', 'wb') as out:
    pickle.dump((hieff, g, gdot), out)

In [22]:
from sympy import symbols, Rational
d, a = symbols('Delta,alpha', real=True)

In [ ]:
frac = simplify(Rational(1, 2) / d**3
                + Rational(-2 - 3 * 2 * 3, 4) / d**2 / (d + a)
                + Rational(2 - 2 * 3, 4) / d / (d + a)**2
                - Rational(2**2, 4) / (d + a)**3
                - Rational(2, 4) / d**2 / (2 * d + a)
                + Rational(9 * 2 * 3, 4) / d**2 / (2 * d + 3 * a))

In [28]:
from sympy import factor
factor(frac.args[3].args[0])

(Delta + alpha)**3*(2*Delta + alpha)*(2*Delta + 3*alpha)

In [31]:
frac2 = simplify(Rational(1, 2) / d**3
                + Rational(-2, 4) / d**2 / (d + a)
                + Rational(2, 4) / d / (d + a)**2
                - Rational(2**2, 4) / (d + a)**3
                - Rational(2, 4) / d**2 / (2 * d + a)
                + Rational(9 * 2 * 3, 4) / d**2 / (2 * d + 3 * a))

In [34]:
simplify(frac - frac2)

3*(-4*Delta - 3*alpha)/(2*Delta**2*(Delta**2 + 2*Delta*alpha + alpha**2))

In [35]:
get_expr_at_order(expansion, int_scale, 4) + sw.gdot_n[4]

-I*[[[G_1,H_I],G_1],G_1]/8 + [[G_1,H_I],G_2]/3 - [[G_1,\dot{G}_2],G_1]/6 + [[G_2,H_I],G_1]/3 - I*[G_1,\dot{G}_3]/2 - I*[G_2,\dot{G}_2]/2 + I*[G_3,H_I]/2

In [37]:
for arg in gdot[3].args:
    if not arg.has(tvar):
        display(arg)